In [15]:
import struct
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import pandas as pd
import csv
import os
#import porn
from os import listdir
from os.path import isfile, join
from pathlib import Path
import math
from collections import Counter
import time

In [2]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)
        
def find2(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return root, name

# create readable structure from txt file
def create_frame_info(fi_name):
    with open(fi_name) as f:
        frame_info = f.readlines()
    return frame_info

# create a list of ids present in a readable structure
def get_ids_from_txt(frame_info):
    ids = []
    for line in frame_info:
        a, rest = line.split(",", 1)
        ids.append(int(a))
    return(ids)

#get set difference
def get_difference(a, b):
    return list(set(a).difference(set(b)))

def remove_rows(dframe, idlist):
    dframe = dframe.set_index("detection_id")
    dframe = dframe.drop(idlist, axis=0)
    return dframe

# get all points from a contour and store as a list of tuples
def get_contour_points(contour):
    cont = contour[2:-1]
    tuples = cont.split(";")
    l = []
    for item in tuples:
        a, b = item.split(" ")
        l.append((int(a), int(b)))
    return l

def process_image(img, x, y, cont_points):
    mask = np.zeros((100,100), dtype=int)
    xdiff = x - 10
    ydiff = y - 10
    xs = []
    ys = []
    for (a,b) in cont_points:
        v = a - xdiff
        u = b - ydiff
        xs.append(u)
        ys.append(v)
        img[u,v] = (0,0,255)
        mask[u,v] = 1
    return img, mask, xs, ys

def normalizedRGB(img):
    newimg=np.zeros((img.shape))
    for i, row in enumerate(img):
        for j, col in enumerate(row):
            b,g,r = col
            s = b+g+r+0.0000000001
            newimg[i,j] = (b/s), (g/s), (r/s)
    return newimg        

def com(img, startpoint, mask):
    (xavg, yavg) = startpoint
    
    nextleft = (xavg, yavg-1)
    nextright = (xavg, yavg+1)
    nextup = (xavg+1, yavg)
    nextdown = (xavg-1, yavg)
    
    if (mask[nextleft]==0):
        mask[nextleft] = 1
        com(img, nextleft, mask)
    if (mask[nextright]==0):
        mask[nextright] = 1
        com(img, nextright, mask)
    if (mask[nextup]==0):
        mask[nextup] = 1
        com(img, nextup, mask)
    if (mask[nextdown]==0):
        mask[nextdown] = 1
        com(img, nextdown, mask)

In [3]:
path_to_csvs = "/home/sam/dissertation/output/hsrgc"
mypath = "/home/sam/dissertation/output/new"

videonames = [f for f in listdir(path_to_csvs) if isfile(join(path_to_csvs, f))]
videonames.remove("clustering_hs.ipynb")

In [4]:
with open("csvnames.txt") as f:
    csvnames = f.readlines()

In [5]:
# static paths to csvs, videos, frame infos, and npys
path_to_csvs = "/media/sam/Sam\'s Drive/SUBSET/SQL/ALLYEARS/"
videos_dir = "/media/sam/My Passport/FISH4KNOWLEDGE/f4k_extracted_image/output/summaries/"
frame_info_dir = "/media/sam/My Passport/FISH4KNOWLEDGE/f4k_extracted_image/output/summaries/"
npy_dir = "/media/sam/My Passport/FISH4KNOWLEDGE/final/"

In [14]:
#videonames: 100e59edcf83bd40956f8f6a7789c6ba#201210060800.csv
#csvnames: data_siteHoBiHu_camera1_201008230800.csv \n
print(len(csvnames))
videonames2 = []
for item in videonames:
    videonames2.append(item[:-4])
print(videonames2[0])

4521
100e59edcf83bd40956f8f6a7789c6ba#201210060800


In [16]:
print("Filename\t\t\t\t\tStatus\t\t% Done\tFrames saved".format())
total = 345
done = 0
index_error = []
others = []
percent = 0
for csvname in csvnames:
    try:
        time.sleep(2)
        saved_imgs = 0
        # format name to get full path to csv
        csv_path = path_to_csvs + csvname[:-1]

        # create initial dataframe
        df = pd.read_csv(csv_path)
        # get video ID from current df
        videoID = df.iloc[0][4][1:]
#         apath = "output/a/"+videoID
#         bpath = "output/b/"+videoID
#         os.mkdir(apath)
#         os.mkdir(bpath)
        # get names for files
        if (videoID in videonames2):
            video_name_fix = "summary_" + videoID + ".avi"
            npy_name_fix = videoID + ".RESULT.npy"
            frame_info_fix = "frame_info_" + videoID + ".txt"

            # get paths to video, frame info and npy
            videopath, videoname = find2(video_name_fix, videos_dir)
            video_path = videopath + "/" + videoname
            frame_info_path = find(frame_info_fix, frame_info_dir)
            npy_path = find(npy_name_fix, npy_dir)

            # get ids from frame info file and from df, remove rows from df that dont exist in frame info
            frame_info_file = create_frame_info(frame_info_path)
            frame_info_ids = get_ids_from_txt(frame_info_file)
            df_ids = df["detection_id"].tolist()
            ids_to_remove = get_difference(df_ids, frame_info_ids)
            clean_df = remove_rows(df, ids_to_remove)

            # load npy file, this is used together with video to produce images
            npy = np.load(npy_path)
            vidcap = cv2.VideoCapture(video_path)
            success,image = vidcap.read()
            # counts every iteration
            count = 0
    #         # initialize df where we will save stats
    #         stats = pd.DataFrame(columns=["fish_id", "specie_id", "hAvg", "sAvg", "nR", "nG", "Compactness"])
    #         stats.index.name = "detection_id"

            while success:
                if (npy[count] and df.iloc[count][' specie_id']==0):
            #         print("Normal image:")
            #         plt.imshow(image)
            #         plt.show()

                    index = str(clean_df.index[count])
                    # save original image
                    imgpath = "output/hsrgc_images/"+videoID+"#"+index+".jpg"
                    cv2.imwrite(imgpath, image)
    #                 contour = clean_df.iloc[count][13]
    #                 x = clean_df.iloc[count][5]
    #                 y = clean_df.iloc[count][6]
    #                 contour_points = get_contour_points(contour)
    #                 cont_img, mask, xs, ys = process_image(image, x, y, contour_points)
    #                 # save image with contour
    #                 contpath = "output/b/"+videoID+"/"+index+".jpg"
    #         #         print("Contoured image:")
    #         #         plt.imshow(cont_img)
    #         #         plt.show()


    #                 a_fish = np.bincount(mask.reshape((10000,)))[1]
    #                 p_fish = len(contour_points)
    #                 r = p_fish / (2*math.pi)
    #                 a_circle = math.pi * r * r
    #                 comp = a_fish / a_circle

    #                 xavg = int(np.mean(xs))
    #                 yavg = int(np.mean(ys))
    #                 # set coordinates for CoM point
    #                 startpoint = (xavg, yavg)

    #                 # compute CoM-based shape
    #                 try:
    #                     com(cont_img, startpoint, mask)            
    # #                     cv2.imwrite(contpath, cont_img)
    #                 except IndexError:
    #                     print("{}\tIndexE\t\t{:2.2f}% left\t{}".format(csvname[:-1], percent, saved_imgs))
    #                     count+= 1
    #                     success, image = vidcap.read()
    #                     continue
    #                 # extract specific channels
    # #                 channel0 = image[:,:,0]
    # #                 channel1 = image[:,:,1]
    # #                 # compute averages
    # #                 c0avg = np.mean(mask*channel0)
    # #                 c1avg = np.mean(mask*channel1)
    #                 # compute normalized image
    #                 normRGB = normalizedRGB(image)
    #                 # extract normalized RGB channels
    #                 normc0 = normRGB[:,:,0]
    #                 normc1 = normRGB[:,:,1]
    #                 # compute averages
    #                 normc0avg = np.mean(mask*normc0)
    #                 normc1avg = np.mean(mask*normc1)
    #                 # save to dataframe

    #                 hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #                 hueChannel = hsv[:,:,0]
    #                 satChannel = hsv[:,:,1]
    #                 havg = np.mean(mask*hueChannel)
    #                 savg = np.mean(mask*satChannel)

    #                 spid = df.iloc[count][' specie_id']
    #                 fid = df.iloc[count][' fish_id']

    #                 stats.loc[index] = [fid, spid, havg, savg, normc0avg, normc1avg, comp]
                    saved_imgs+=1
                count+= 1
                success, image = vidcap.read()

#         csvpath = "output/hsrgc_unknown/"+videoID+".csv"
#         stats.to_csv(csvpath)
        done+=1
        percent = done / total
        print("{}\tDone\t\t{:2.2f}% left\t{}".format(csvname[:-1], percent, saved_imgs))
        
    except Exception as e:
        print("{}\t{}".format(csvname[:-1], e))
        others.append(csvname)
        time.sleep(2)
        continue

Filename					Status		% Done	Frames saved
data_siteHoBiHu_camera1_201008230800.csv	Done		0.00% left	0
data_siteHoBiHu_camera1_201008240800.csv	Done		0.01% left	5
data_siteHoBiHu_camera1_201008260800.csv	Done		0.01% left	25
data_siteHoBiHu_camera1_201008270800.csv	Done		0.01% left	19
data_siteHoBiHu_camera1_201008280800.csv	Done		0.01% left	16
data_siteHoBiHu_camera1_201008290800.csv	Done		0.02% left	60
data_siteHoBiHu_camera1_201009020800.csv	Done		0.02% left	11
data_siteHoBiHu_camera1_201009030800.csv	Done		0.02% left	38
data_siteHoBiHu_camera1_201009040800.csv	Done		0.03% left	34
data_siteHoBiHu_camera1_201009060800.csv	Done		0.03% left	1
data_siteHoBiHu_camera1_201009070800.csv	Done		0.03% left	28
data_siteNPP-3_camera1_201203290800.csv	Done		0.03% left	0
data_siteNPP-3_camera1_201203310800.csv	Done		0.04% left	0
data_siteNPP-3_camera1_201204020800.csv	Done		0.04% left	0
data_siteNPP-3_camera1_201204040800.csv	Done		0.04% left	0
data_siteNPP-3_camera1_201204060800.csv	Done		0.05% lef

data_siteNPP-3_camera3_201108210800.csv	Done		0.40% left	0
data_siteNPP-3_camera3_201108230800.csv	Done		0.40% left	0
data_siteNPP-3_camera3_201108260800.csv	Done		0.41% left	0
data_siteNPP-3_camera3_201108270800.csv	Done		0.41% left	92
data_siteNPP-3_camera3_201108280800.csv	Done		0.41% left	84
data_siteNPP-3_camera3_201108290800.csv	Done		0.41% left	33
data_siteNPP-3_camera3_201108300800.csv	Done		0.42% left	0
data_siteNPP-3_camera3_201109090800.csv	Done		0.42% left	0
data_siteNPP-3_camera1_201307100800.csv	Done		0.42% left	341
data_siteNPP-3_camera2_200910010800.csv	Done		0.43% left	22
data_siteNPP-3_camera2_200910030800.csv	Done		0.43% left	21
data_siteNPP-3_camera2_200910060800.csv	Done		0.43% left	7
data_siteNPP-3_camera2_200910070800.csv	Done		0.43% left	247
data_siteNPP-3_camera2_200910080800.csv	Done		0.44% left	3
data_siteNPP-3_camera2_200910090800.csv	Done		0.44% left	20
data_siteNPP-3_camera2_200910150800.csv	Done		0.44% left	466
data_siteNPP-3_camera2_200910160800.csv	Done

data_siteNPP-3_camera1_201108100800.csv	Done		0.80% left	0
data_siteNPP-3_camera1_201108110800.csv	Done		0.80% left	0
data_siteNPP-3_camera1_201108120800.csv	Done		0.80% left	0
data_siteNPP-3_camera1_201108130800.csv	Done		0.81% left	1068
data_siteHoBiHu_camera1_201102120800.csv	Done		0.81% left	0
data_siteHoBiHu_camera1_201102130800.csv	Done		0.81% left	0
data_siteHoBiHu_camera1_201102140800.csv	Done		0.81% left	272
data_siteHoBiHu_camera1_201102150800.csv	Done		0.82% left	0
data_siteHoBiHu_camera1_201102160800.csv	Done		0.82% left	0
data_siteHoBiHu_camera1_201102180800.csv	Done		0.82% left	0
data_siteHoBiHu_camera1_201102190800.csv	Done		0.83% left	0
data_siteHoBiHu_camera1_201102200800.csv	Done		0.83% left	0
data_siteHoBiHu_camera1_201102210800.csv	Done		0.83% left	109
data_siteHoBiHu_camera1_201102220800.csv	Done		0.83% left	0
data_siteHoBiHu_camera1_201102230800.csv	Done		0.84% left	0
data_siteLanYu_camera2_201104290800.csv	Done		0.84% left	0
data_siteLanYu_camera2_201105010800.cs

data_siteLanYu_camera1_201104090800.csv	Done		1.19% left	69
data_siteLanYu_camera1_201104110800.csv	Done		1.20% left	195
data_siteNPP-3_camera3_201204250800.csv	Done		1.20% left	0
data_siteNPP-3_camera3_201204260800.csv	Done		1.20% left	0
data_siteNPP-3_camera3_201204280800.csv	Done		1.21% left	0
data_siteNPP-3_camera3_201204300800.csv	Done		1.21% left	0
data_siteNPP-3_camera3_201205010800.csv	Done		1.21% left	0
data_siteNPP-3_camera3_201205020800.csv	Done		1.21% left	0
data_siteNPP-3_camera3_201205030800.csv	Done		1.22% left	0
data_siteNPP-3_camera3_201205040800.csv	Done		1.22% left	0
data_siteNPP-3_camera3_201205050800.csv	Done		1.22% left	0
data_siteNPP-3_camera3_201205060800.csv	Done		1.23% left	0
data_siteNPP-3_camera3_201205070800.csv	Done		1.23% left	0
data_siteNPP-3_camera3_201205080800.csv	Done		1.23% left	0
data_siteNPP-3_camera3_201205090800.csv	Done		1.23% left	0
data_siteNPP-3_camera2_201304170800.csv	Done		1.24% left	544
data_siteNPP-3_camera2_201304190800.csv	Done		1.24%

data_siteHoBiHu_camera2_201010080800.csv	Done		1.60% left	0
data_siteHoBiHu_camera2_201010090800.csv	Done		1.60% left	0
data_siteHoBiHu_camera2_201010100800.csv	Done		1.60% left	0
data_siteHoBiHu_camera2_201010110800.csv	Done		1.61% left	0
data_siteHoBiHu_camera2_201010120800.csv	Done		1.61% left	0
data_siteHoBiHu_camera2_201010130800.csv	Done		1.61% left	0
data_siteHoBiHu_camera2_201010140800.csv	Done		1.61% left	0
data_siteHoBiHu_camera2_201010160800.csv	Done		1.62% left	0
data_siteNPP-3_camera3_201101070800.csv	Done		1.62% left	0
data_siteNPP-3_camera3_201101080800.csv	Done		1.62% left	0
data_siteNPP-3_camera3_201101090800.csv	Done		1.63% left	0
data_siteNPP-3_camera3_201101100800.csv	Done		1.63% left	0
data_siteNPP-3_camera3_201101110800.csv	Done		1.63% left	0
data_siteNPP-3_camera3_201101120800.csv	Done		1.63% left	0
data_siteNPP-3_camera3_201101130800.csv	Done		1.64% left	0
data_siteNPP-3_camera3_201101140800.csv	Done		1.64% left	0
data_siteNPP-3_camera3_201101170800.csv	Done		1.

...

data_siteNPP-3_camera2_201112140800.csv	Done		11.24% left	0
data_siteNPP-3_camera2_201112160800.csv	Done		11.24% left	0
data_siteNPP-3_camera2_201112170800.csv	Done		11.25% left	0
data_siteNPP-3_camera2_201112180800.csv	Done		11.25% left	0
data_siteNPP-3_camera2_201112200800.csv	Done		11.25% left	0
data_siteNPP-3_camera2_201201190800.csv	Done		11.26% left	0
data_siteNPP-3_camera2_201101090800.csv	Done		11.26% left	0
data_siteNPP-3_camera2_201101100800.csv	Done		11.26% left	0
data_siteNPP-3_camera2_201101110800.csv	Done		11.26% left	0
data_siteNPP-3_camera2_201101120800.csv	Done		11.27% left	0
data_siteNPP-3_camera2_201101130800.csv	Done		11.27% left	0
data_siteNPP-3_camera2_201101140800.csv	Done		11.27% left	0
data_siteNPP-3_camera2_201101150800.csv	Done		11.28% left	0
data_siteNPP-3_camera2_201101160800.csv	Done		11.28% left	0
data_siteNPP-3_camera2_201101170800.csv	Done		11.28% left	0
data_siteNPP-3_camera2_201101180800.csv	Done		11.28% left	0
data_siteNPP-3_camera2_201101190800.csv	

data_siteNPP-3_camera4_201106090800.csv	Done		11.64% left	0
data_siteNPP-3_camera4_201106100800.csv	Done		11.64% left	0
data_siteNPP-3_camera1_201106030800.csv	Done		11.64% left	0
data_siteNPP-3_camera1_201106050800.csv	Done		11.65% left	0
data_siteNPP-3_camera1_201106060800.csv	Done		11.65% left	0
data_siteNPP-3_camera1_201106070800.csv	Done		11.65% left	0
data_siteNPP-3_camera1_201106080800.csv	Done		11.66% left	0
data_siteNPP-3_camera1_201106090800.csv	Done		11.66% left	0
data_siteNPP-3_camera1_201106100800.csv	Done		11.66% left	0
data_siteNPP-3_camera1_201106110800.csv	Done		11.66% left	0
data_siteNPP-3_camera1_201106120800.csv	Done		11.67% left	0
data_siteNPP-3_camera1_201106130800.csv	Done		11.67% left	0
data_siteNPP-3_camera1_201106140800.csv	Done		11.67% left	0
data_siteNPP-3_camera1_201109020800.csv	Done		11.68% left	0
data_siteNPP-3_camera1_201109090800.csv	Done		11.68% left	0
data_siteNPP-3_camera1_201109110800.csv	Done		11.68% left	0
data_siteNPP-3_camera1_201109120800.csv	

data_siteNPP-3_camera4_201210310800.csv	Done		12.03% left	0
data_siteNPP-3_camera4_201211010800.csv	Done		12.04% left	0
data_siteNPP-3_camera4_201211020800.csv	Done		12.04% left	0
data_siteNPP-3_camera4_201211030800.csv	Done		12.04% left	0
data_siteNPP-3_camera4_201211040800.csv	Done		12.05% left	0
data_siteNPP-3_camera4_201211050800.csv	Done		12.05% left	0
data_siteNPP-3_camera4_201211060800.csv	Done		12.05% left	0
data_siteNPP-3_camera3_201003260800.csv	Done		12.06% left	0
data_siteNPP-3_camera3_201003270800.csv	Done		12.06% left	0
data_siteNPP-3_camera3_201003280800.csv	Done		12.06% left	0
data_siteNPP-3_camera3_201003290800.csv	Done		12.06% left	0
data_siteNPP-3_camera3_201003300800.csv	Done		12.07% left	0
data_siteNPP-3_camera3_201003310800.csv	Done		12.07% left	0
data_siteNPP-3_camera3_201004010800.csv	Done		12.07% left	0
data_siteNPP-3_camera3_201004020800.csv	Done		12.08% left	0
data_siteNPP-3_camera3_201004030800.csv	Done		12.08% left	0
data_siteNPP-3_camera3_201004040800.csv	

data_siteLanYu_camera1_201005130800.csv	Done		12.43% left	0
data_siteLanYu_camera1_201005140800.csv	Done		12.43% left	0
data_siteLanYu_camera1_201005150800.csv	Done		12.44% left	0
data_siteLanYu_camera1_201005200800.csv	Done		12.44% left	0
data_siteLanYu_camera1_201005220800.csv	Done		12.44% left	0
data_siteHoBiHu_camera1_201009090800.csv	Done		12.45% left	0
data_siteHoBiHu_camera1_201009110800.csv	Done		12.45% left	0
data_siteHoBiHu_camera1_201009120800.csv	Done		12.45% left	0
data_siteHoBiHu_camera1_201009130800.csv	Done		12.46% left	0
data_siteHoBiHu_camera1_201009150800.csv	Done		12.46% left	0
data_siteHoBiHu_camera1_201009160800.csv	Done		12.46% left	0
data_siteHoBiHu_camera1_201009170800.csv	Done		12.46% left	0
data_siteHoBiHu_camera1_201009190800.csv	Done		12.47% left	0
data_siteHoBiHu_camera1_201009210800.csv	Done		12.47% left	0
data_siteNPP-3_camera2_201002150800.csv	Done		12.47% left	0
data_siteNPP-3_camera2_201002160800.csv	Done		12.48% left	0
data_siteNPP-3_camera2_20100219

KeyboardInterrupt: 

In [20]:
img = cv2.imread("432487764.jpg")
img.shape

(100, 100, 3)

In [11]:
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [36]:
dummy = np.zeros((100,100))
img = cv2.imread("432487764.jpg")

In [82]:
nor = normalizedRGB(img)

/home/sam/anaconda3/envs/mlp/lib/python3.7/site-packages/ipykernel_launcher.py:64: RuntimeWarning: overflow encountered in ubyte_scalars


In [93]:
mask = np.reshape(mask, mask.size)
orig = np.reshape(mask, (100,100))
nR = nor[:,:,0].reshape((10000,))
mult = mask * nR
for item in mult:
    if (item!=0.0):
        print(item)

11.238095238041724
10.818181818132645
10.818181818132645
9.999999999958334
9.30769230765651
9.3461538461179
8.74999999996875
8.857142857111224
8.857142857111224
8.333333333305555
8.857142857111224
8.857142857111224
9.111111111077367
9.75999999996096
9.75999999996096
11.52380952375465
12.421052631513575
12.421052631513575
12.421052631513575
12.421052631513575
12.421052631513575
12.421052631513575
11.238095238041724
11.238095238041724
10.818181818132645
10.818181818132645
9.999999999958334
9.30769230765651
9.3461538461179
8.74999999996875
8.857142857111224
8.857142857111224
8.333333333305555
8.857142857111224
8.857142857111224
9.111111111077367
9.75999999996096
10.521739130389037
11.476190476135828
13.277777777704014
12.421052631513575
12.421052631513575
12.421052631513575
12.421052631513575
12.421052631513575
12.421052631513575
11.238095238041724
11.238095238041724
10.818181818132645
10.818181818132645
9.999999999958334
9.30769230765651
9.3461538461179
8.74999999996875
8.857142857111224

In [62]:
1+2

3